# Training With Context

## !! Context Training is not working properly in this version !!

## Load Packages

In [4]:
import torch
from LSTM_context import *
import torch.nn as nn
import Tmaze_Context_2 as TMC
import plotting_functions as pf
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

importlib.reload(pf)

# Check CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.6.0+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060


## Device Helpers

In [5]:
def ensure_same_device(model, *tensors):
    """Moves all tensors to the same device as the model."""
    device = next(model.parameters()).device
    return [t.to(device) for t in tensors]

def print_device_info(*tensors, model=None):
    """Prints device information for debugging."""
    if model:
        print(f"Model device: {next(model.parameters()).device}")
    for i, t in enumerate(tensors):
        print(f"Tensor {i} device: {t.device}")

## Train on Multi/Uni with context and trial counter

In [6]:
importlib.reload(TMC)

# Set device and print for verification
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

tmaze = TMC.TmazeEnvContext()
# Generate trials with context and trial counter
u, z, mask, conditions_train = tmaze.generate_trials(
    n_trials=10000,
    alpha=0.2,
    sigma_in=0.01,
    baseline=0.2,
    n_coh=15,
    trials_per_context=50,
    modality='multi',
    add_context=True,  # Explicitly add context input
    add_trial_counter=True  # Explicitly add trial counter input
)

# No need to convert to tensor again as generate_trials already returns tensors
u = u.to(device)
z = z.to(device)
mask = mask.to(device)

# Create and move model to GPU
input_size = u.shape[2]  
net = Net_lstm(n=100, input_size=input_size, dale=True, sigma_rec=0.15)
criterion = nn.CrossEntropyLoss()
net.lstm.flatten_parameters()
net = net.to(device)

# Verify device placement
print(f"Model device: {next(net.parameters()).device}")
print(f"Input tensor device: {u.device}")
print(f"Target tensor device: {z.device}")
print(f"Mask tensor device: {mask.device}")

# Train the network
net.fit(u, z, mask, 
        lr=0.0001,  # Learning rate for unimodal training
        epochs=300, 
        verbose=True, 
        weight_decay=0.001)

# Print metrics
print(f"\nInput tensor shape: {u.shape}")
print(f"Target tensor shape: {z.shape}")
print(f"Mask tensor shape: {mask.shape}")

# Count trials by context
visual_count = sum(1 for condition in conditions_train if condition['context'] == 'visual')
audio_count = sum(1 for condition in conditions_train if condition['context'] == 'audio')

print(f"\nVisual trials: {visual_count}")
print(f"Audio trials: {audio_count}")


Using device: cuda
Model device: cuda:0
Input tensor device: cuda:0
Target tensor device: cuda:0
Mask tensor device: cuda:0


c:\Users\Runyan1\AppData\Local\anaconda3\envs\RNN\Lib\site-packages\torch\nn\modules\rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:1412.)
  result = _VF.lstm(


Epoch: 5/300............. mse_z: 0.0663
Epoch: 10/300............. mse_z: 0.0610
Epoch: 15/300............. mse_z: 0.0564
Epoch: 20/300............. mse_z: 0.0464
Epoch: 25/300............. mse_z: 0.0420
Epoch: 30/300............. mse_z: 0.0409
Epoch: 35/300............. mse_z: 0.0383
Epoch: 40/300............. mse_z: 0.0281
Epoch: 45/300............. mse_z: 0.0120
Epoch: 50/300............. mse_z: 0.0082
Epoch: 55/300............. mse_z: 0.0069
Epoch: 60/300............. mse_z: 0.0060
Epoch: 65/300............. mse_z: 0.0059
Epoch: 70/300............. mse_z: 0.0053
Epoch: 75/300............. mse_z: 0.0054
Epoch: 80/300............. mse_z: 0.0060
Epoch: 85/300............. mse_z: 0.0051
Epoch: 90/300............. mse_z: 0.0048
Epoch: 95/300............. mse_z: 0.0056
Epoch: 100/300............. mse_z: 0.0045
Epoch: 105/300............. mse_z: 0.0044
Epoch: 110/300............. mse_z: 0.0042
Epoch: 115/300............. mse_z: 0.0045
Epoch: 120/300............. mse_z: 0.0042
Epoch: 125/3

## Check inputs

## Save trained model